## Introduction
Computed herein are the Elastic Spectra for horizontal and vertical seismic loading in accordance with draft NZS 1170.5 [2024]. This is a unverified and uncontrolled document.

Please verify before any serious use. I'll not take any responsibility for correctness.

This is also an example of basic python programming.


## Import python packages

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from aquarel import load_theme
from pathlib import Path
from typing import Union
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interactive
%matplotlib inline

In [8]:
hazard_data_folder = Path.cwd() / "draft_TS_1170_hazard_data"

## Define the spectral acceleration function

The function below computes elastic spectra for horizontal loadings only in accordance with draft NZS TS1170.5.

$$
S_a(T) = \begin{cases}
pga + (S_{a,s}-pga) \cdot \frac{T}{T_B} & \text{for } T \leq T_B \\
S_{a,s} & \text{for } T_B < T \leq T_C \\
S_{a,s} \cdot \left(\frac{T_C}{T}\right) & \text{for } T_C < T \leq T_D \\
S_{a,s} \cdot \left(\frac{T_C}{T}\right) \cdot \left(\frac{T_D}{T}\right)^{0.50} & \text{for } T_C < T \\
\end{cases}
$$

where,

$pga = $ peak ground acceleration (g)

$S_{a,s} = $ Short period spectral acceleration (g)

$T_B = $ Lower limit of spectral acceleration plateau (s) = 0.10 s (default)

$T_C = $ Upper limit of spectral acceleration plateau (s)

$T_D = $ Lower limit of spectral displacement plateau (s) = 3.0 s (default)

In [9]:
def define_TS1170_spectra(pga: float = 1.79,
                          Sas: float = 3.85,
                          fault_D: Union[float, str] = 0.0,
                          Tc: float = 0.40,
                          Tb: float = 0.10,
                          Td: float = 3.0,
                          Tmax: float = 5.0) -> np.ndarray:

    """
    Computes spectral acceleration and displacement for horizontal loading
    using draft NZS TS1170.5.

    Args:
        pga (float, optional): Peak ground acceleration. Defaults to 1.79.
        Sas (float, optional): Short period spectral acceleration. Defaults to 3.85.
        fault_D (float or str, optional): Fault distance. Defaults to 0.0.
        Tc (float, optional): Start of velocity plateau. Defaults to 0.40.
        Tb (float, optional): Start of acceleration plateau. Defaults to 0.10.
        Td (float, optional): Start of displacement plateau. Defaults to 3.0.
        Tmax (float, optional): Maximum period. Defaults to 5.

    Returns:
        numpy.ndarray: A 2D array with columns [Tn, psa, sd].
    """

    # Compute time period as a numpy array
    Tn = np.linspace(0.0, Tmax, 100)

    # Compute spectral acceleration
    Sa = pga + (Sas - pga) * (Tn/Tb)
    Sa[Tn > Tb] = Sas
    Sa[Tn > Tc] = Sas * Tc / Tn[Tn > Tc]
    Sa[Tn > Td] = Sas * (Tc / Tn[Tn > Td]) * (Td / Tn[Tn > Td]) ** 0.5

    # Compute max near-fault factor as a function of period
    Nmax = np.maximum(1.0, 0.24 * Tn + 0.64)
    Nmax[Tn > 4.0] = np.minimum(1.72, 0.12 * Tn[Tn > 4.0] + 1.12)

    # Compute near-fault factor
    if isinstance(fault_D, str) and (fault_D == ">20"):
        N = np.ones_like(Tn)

    elif isinstance(fault_D, (int, float)) and np.isnan(fault_D):
        N = np.ones_like(Tn)

    else:
        fault_D = float(fault_D)
        N = np.where(fault_D <= 2.0, Nmax, 1 + (Nmax - 1) * (20 - fault_D) / 18.0)

    # Compute spectral acceleration and displacement
    psa = Sa * N
    sd = Tn**2 * Sa * 9.807 / (2 * np.pi)**2

    return np.array([Tn, psa, sd]).T

In [10]:
def plot_spectra(spectra) :
    """Plots spectral acceleration and displacement as a function of period using plotly.

    Args:
        spectra (numpy.ndarray): A 2D array with columns [Tn, psa, sd].
    """
    
    # Error handling: Check if spectra has the correct shape
    if spectra.shape[1] != 3:
        raise ValueError("Spectra should be a 2D array with 3 columns [Tn, psa, sd].")
        
    #theme = load_theme("arctic_dark")
    #theme.apply()
        
    # Create a subplot with 1 row and 2 columns
    fig, axs = plt.subplots(1, 2, figsize=(15, 6))

    # Add spectral acceleration plot
    axs[0].plot(spectra[:, 0], spectra[:, 1], label='Spectral Acceleration', linewidth=2)
    axs[0].set_xlabel('Period (s)')
    axs[0].set_ylabel('Spectral Acceleration (g)')
    axs[0].legend()
    axs[0].grid(True, which='major', linestyle='-', linewidth=0.75)
    axs[0].grid(True, which='minor', linestyle='--', linewidth=0.5)
    axs[0].minorticks_on()


    # Add spectral displacement plot
    axs[1].plot(spectra[:, 0], spectra[:, 2], label='Spectral Displacement', linewidth=2)
    axs[1].set_xlabel('Period (s)')
    axs[1].set_ylabel('Spectral Displacement (m)')
    axs[1].legend()
    axs[1].grid(True, which='major', linestyle='-', linewidth=0.75)
    axs[1].grid(True, which='minor', linestyle='--', linewidth=0.5)
    axs[1].minorticks_on()
        
    # Show the plot
    plt.show()  
    #theme.apply_transforms()

# Define the interactive function
def interactive_plot(func, **kwargs):
    interact_obj = interactive(func, **kwargs)
    return interact_obj

    

In [11]:
# Read the list of locations available
locations = pd.read_csv(hazard_data_folder / "locations.csv")
locations = locations.T.values.flatten().tolist()

# Read the list of return periods and site classes available
return_periods = [25, 50, 100, 250, 500, 1000, 2500]
site_classes = ['I', 'II', 'III', 'IV', 'V', 'VI', 'default']


def process_data(location, site_class, return_period) :

    # Read hazard data
    hazard_file = hazard_data_folder / f"TS1170_hazard_apoe_{return_period}.csv"
    hazard_data = pd.read_csv(hazard_file)

    # Get the index of the row with the matching location
    index = hazard_data[hazard_data["location"] == location].index[0]

    # Access hazard values for various site classes
    eqM, fault_D = hazard_data.loc[index, ['M', 'D']]    

    if site_class in ('I', 'II', 'III', 'IV', 'V', 'VI') :

        pga, Sas = hazard_data.loc[index, [f'{site_class}-PGA', f'{site_class}-Sas']]
        Tc = hazard_data.loc[index, f'{site_class}-Tc']

        # Compute spectral acceleration and displacement for one site class
        spectra_TS1170 = define_TS1170_spectra(fault_D = fault_D,
                                               pga = pga,
                                               Sas = Sas,
                                               Tc = Tc,
                                               Tmax = 5,
                                               )
        
    else :

        spectra = {}
        for site in ('II', 'III', 'IV', 'V') :

            pga, Sas = hazard_data.loc[index, [f'{site}-PGA', f'{site}-Sas']]
            Tc = hazard_data.loc[index, f'{site}-Tc']

            spectra[site] = define_TS1170_spectra(fault_D = fault_D,
                                               pga = pga,
                                               Sas = Sas,
                                               Tc = Tc,
                                               Tmax = 5,
                                               )
        spectra_stack = np.stack(list(spectra.values()), axis=-1)
            
        spectra_TS1170 = np.max(spectra_stack, axis = -1)
        


    print(f'------------------------------------------')
    print(f'Site location : {location}')
    print(f'Design return period : {return_period} years')
    print(f'Design earthquake magnitude M : {eqM}')
    print(f'Distance to major fault D : {fault_D} km')
    print(f'Peak ground acceleration PGA : {pga} g')
    print(f'Short period spectral acceleration Sas: {Sas} g')
    print(f'Spectral acceleration plateau Tc: {Tc} s')
    print(f'------------------------------------------')

    
    
    # print(spectra_TS1170)

    # Plot the spectra
    plot_spectra(spectra_TS1170)

In [12]:
# Use interact function
interact_plot = interactive_plot(process_data,
         location=widgets.Dropdown(options=locations, value="Wellington CBD", description="Site location:"),
         site_class=widgets.Dropdown(options=site_classes, value="V", description="Site class:"),
         return_period=widgets.Dropdown(options=return_periods, value=500, description="Return Period:")
        )

display(interact_plot)

interactive(children=(Dropdown(description='Site location:', index=134, options=('Kaitaia', 'Kerikeri', 'Harur…